In [1]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np

def read_basic_data():
    # Excel file names
    travel_times = pd.read_excel('a2_part1.xlsx','Travel Times')
    lines = pd.read_excel('a2_part1.xlsx', 'Lines')

    lines = lines.drop(['Name','Frequency'], axis = 1)
    lines = lines.fillna(0)

    weights = 1 #Each activity has the same importance for us
    
    return travel_times, lines, weights

travel_times, lines, weights = read_basic_data()

In [7]:

def build_model(travel_times, lines, weights):

    # Initialize the Gurobi model
    model = Model("NS_trains")

    line_names = ['800','3000','3100','3500','3900']
    runnning_activites = {}
    dwelling_activities = {}
    transfer_activities = {}
    sync_activities = {}
    headway_activities = {}
    events_pi = {}

    for k in range(5):
        station_names = lines.iloc[k].tolist()  # Extract station names from the first row
        name = line_names[k]

        # Create Gurobi variables for each possible trip between stations
        for i in range(len(station_names) - 1):
            station1 = str(station_names[i])
            station2 = str(station_names[i+1])

            if station1 == '0' or station2 == '0':
                break
            # Create a Gurobi variable for the activity between station1 and station2
            runnning_activites[f'{station1} - {station2} - {name}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                        name=f"activity_{station1}{station2}{name}")
            runnning_activites[f'{station2} - {station1} - {name}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                        name=f"activity_{station2}{station1}{name}")
            

            if i != 0 and i != len(station_names):
                dwelling_activities[f'dwelling - {station1} - {name} forward trip'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f"activity_dwelling at {station1} - {name} forward trip")
                dwelling_activities[f'dwelling - {station1} - {name} backward trip'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f"activity_dwelling at {station1} - {name} backward trip")
        

# Events
            events_pi[f'arrival - {station1} - {station2} - {name}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                            name=f"arrival_event_{station1}{station2}{name}")
            if i != len(station_names):
                events_pi[f'departure - {station1} - {station2} - {name}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                            name=f"departure_event_{station1}{station2}{name}")
                
            
    # Four Transfer activities at Eindhoven           
    transfer_activities[f'transfer Hrl - Ut - 3500'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f"activity_transfer at Hrl - Ut - 3500")
    transfer_activities[f'transfer Hrl - Ut - 800'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f"activity_transfer at Hrl - Ut - 800 ")
    transfer_activities[f'transfer Ut - Hrl - 800'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f"activity_transfer at Ut - Hrl - 800 ")
    transfer_activities[f'transfer Ut - Hrl - 3500'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f"activity_transfer at Ut - Hrl - 3500")
    
    # Headway at Utrecht from Ams/Sch and to Ams/Sch
    headway_activities = ...
    
    
    
    
    
    
    
    
    
    
    # Objective function: Minimize the total weighted travel time
    objective = quicksum((rav * weights for rav in runnning_activites.values()))
    
    
    model.setObjective(objective, GRB.MINIMIZE)
    model.update()
    
    return model

model = build_model(travel_times, lines, weights)

{'arrival - Amr - Asd - 800': <gurobi.Var *Awaiting Model Update*>, 'departure - Amr - Asd - 800': <gurobi.Var *Awaiting Model Update*>, 'arrival - Asd - Ut - 800': <gurobi.Var *Awaiting Model Update*>, 'departure - Asd - Ut - 800': <gurobi.Var *Awaiting Model Update*>, 'arrival - Ut - Ehv - 800': <gurobi.Var *Awaiting Model Update*>, 'departure - Ut - Ehv - 800': <gurobi.Var *Awaiting Model Update*>, 'arrival - Ehv - Std - 800': <gurobi.Var *Awaiting Model Update*>, 'departure - Ehv - Std - 800': <gurobi.Var *Awaiting Model Update*>, 'arrival - Std - Mt - 800': <gurobi.Var *Awaiting Model Update*>, 'departure - Std - Mt - 800': <gurobi.Var *Awaiting Model Update*>}
{'arrival - Amr - Asd - 800': <gurobi.Var *Awaiting Model Update*>, 'departure - Amr - Asd - 800': <gurobi.Var *Awaiting Model Update*>, 'arrival - Asd - Ut - 800': <gurobi.Var *Awaiting Model Update*>, 'departure - Asd - Ut - 800': <gurobi.Var *Awaiting Model Update*>, 'arrival - Ut - Ehv - 800': <gurobi.Var *Awaiting Mode

KeyboardInterrupt: 